# PROYECTO FINAL GRUPAL  - NBA-StatVision - Lesiones


### Importacion de librerias


In [1]:
import pandas as pd 
import numpy as np 


Importamos dataframe "play_by_play" pero solo leemos la primer fila que es el nombre de la columna


In [2]:
## creamos variable para alojar la ruta del archivo
ruta_archivo_csv = r'C:\Users\Fernando\Desktop\CSV de NBA\archive\csv\play_by_play.csv'

# df = pd.read_csv(ruta_archivo_csv)

## Leemos solo la primer fila con el nombre de las columnas.
def obtener_nombre_columnas(ruta_archivo_csv):
    
    columnas = pd.read_csv(ruta_archivo_csv, nrows=0).columns.tolist()
    return columnas




Vemos los nombres de las columnas de "play_by_play" (esto es porque nos trae la primer fila del dataset)

In [3]:
primera_fila = obtener_nombre_columnas(ruta_archivo_csv)
# mostramos las columnas:
print("Columnas del archivo 'play_by_play': ")
for col in primera_fila:
    print("-", col)

Columnas del archivo 'play_by_play': 
- game_id
- eventnum
- eventmsgtype
- eventmsgactiontype
- period
- wctimestring
- pctimestring
- homedescription
- neutraldescription
- visitordescription
- score
- scoremargin
- person1type
- player1_id
- player1_name
- player1_team_id
- player1_team_city
- player1_team_nickname
- player1_team_abbreviation
- person2type
- player2_id
- player2_name
- player2_team_id
- player2_team_city
- player2_team_nickname
- player2_team_abbreviation
- person3type
- player3_id
- player3_name
- player3_team_id
- player3_team_city
- player3_team_nickname
- player3_team_abbreviation
- video_available_flag


Este data set no tiene datos de fecha, solo 'game_id', entonces deberemos buscar en otro dataset, si hay game_id con fechas.

# ______________________________________________________________________-

Buscamos en el dataset 'game_info' para ver si contiene una columna de fechas.

In [37]:
## Definimos la ruta donde se encuentra
ruta_game_info = r'C:\Users\Fernando\Desktop\CSV de NBA\archive\csv\game_info.csv' 

## Leemos el dataset
df = pd.read_csv(ruta_game_info)

## Lo mostramos 
df.head(2000)


,game_id,game_date,attendance,game_time
0,24600001,1946-11-01,NaN,NaN
1,24600003,1946-11-02,NaN,NaN
2,24600002,1946-11-02,NaN,NaN
3,24600004,1946-11-02,NaN,NaN
4,24600005,1946-11-02,NaN,NaN
...,...,...,...,...
1995,25200088,1952-12-05,NaN,NaN
1996,25200090,1952-12-06,NaN,NaN
1997,25200092,1952-12-06,NaN,NaN
1998,25200091,1952-12-06,NaN,NaN


Como tiene columna de fechas, vamos a filtrar los id que estén entre 2013 y 2023

In [39]:
# Cargamos nuevamente el data set, para "parsear" la columna 'game_date'.
df_games = pd.read_csv(ruta_game_info, parse_dates=['game_date'])

# Convertimos al formato fecha.
# df_games['game_date'] = pd.to_datetime(df_games['game_date'])
df_games['game_date'] = pd.to_datetime(df_games['game_date'], format='%Y-%m-%d')

print(df_games['game_date'].dtype)


# Filtrar por fechas:
games_filtred = df_games [
    (df_games['game_date'] >= '2012-10-30') &
    (df_games['game_date'] <= '2023-12-06')
]

# obtener el set de 'game_id' válidos
game_ids_validos = set(games_filtred['game_id'])
print(f'Cantidad de partidos entre 2013 y 2023: {len(game_ids_validos)}')

# Muestro los tipos de datos:
for columna in df_games.columns:
    print(columna)
    print(df_games[columna].describe())
    print('------------------------------------')




datetime64[ns]
Cantidad de partidos entre 2013 y 2023: 12405
game_id
count    5.805300e+04
mean     2.579790e+07
std      6.331314e+06
min      1.050000e+07
25%      2.130028e+07
50%      2.590030e+07
75%      2.880075e+07
max      4.980009e+07
Name: game_id, dtype: float64
------------------------------------
game_date
count                            58053
mean     1995-08-05 03:39:01.646426496
min                1946-11-01 00:00:00
25%                1983-04-24 00:00:00
50%                1998-02-17 00:00:00
75%                2010-11-26 00:00:00
max                2023-06-12 00:00:00
Name: game_date, dtype: object
------------------------------------
attendance
count     52673.000000
mean      15097.689158
std        5142.654175
min           0.000000
25%       12260.000000
50%       16205.000000
75%       18700.000000
max      200049.000000
Name: attendance, dtype: float64
------------------------------------
game_time
count     29942
unique      144
top        2:13
freq       130

Guardo los cambios en el archivo

In [40]:
df_games.to_csv(r'C:\Users\Fernando\Desktop\CSV de NBA\archive\csv\game_info.csv', index=False)

# _________________________________________________________________________________________

 Uso de Chunksize para filtrar el dataset de 'play_by_play' con los 'game_id' obtenidos de 'game_info'

In [11]:
# Creamos una lista para los Chunks

chunks = []

for chunk in pd.read_csv(r'C:\Users\Fernando\Desktop\CSV de NBA\archive\csv\play_by_play.csv', chunksize=100000):
    chunk_filtrado = chunk[chunk['game_id'].isin(game_ids_validos)]
    chunks.append(chunk_filtrado) ## llena la lista 

# Concatenamos los chunks
play_by_play_filtrado = pd.concat(chunks, ignore_index=True)

## mostramos el dataset filtrado
print(play_by_play_filtrado.shape)
print(f'Cantidad de juegos entre 2013 y 2023: {len(play_by_play_filtrado)}')
play_by_play_filtrado.head()





(5228811, 34)
Cantidad de juegos entre 2013 y 2023: 5228811


,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,wctimestring,pctimestring,homedescription,neutraldescription,visitordescription,...,player2_team_nickname,player2_team_abbreviation,person3type,player3_id,player3_name,player3_team_id,player3_team_city,player3_team_nickname,player3_team_abbreviation,video_available_flag
0,41200161,0,12,0,1,5:41 PM,12:00,NaN,Start of 1st Period (5:41 PM EST),NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
1,41200161,1,10,0,1,5:41 PM,12:00,Jump Ball Koufos vs. Bogut: Tip to Lawson,NaN,NaN,...,Warriors,GSW,4.0,201951,Ty Lawson,1.610613e+09,Denver,Nuggets,DEN,0
2,41200161,2,2,42,1,5:42 PM,11:40,MISS Fournier Driving Layup,NaN,Bogut BLOCK (1 BLK),...,NaN,NaN,5.0,101106,Andrew Bogut,1.610613e+09,Golden State,Warriors,GSW,0
3,41200161,3,4,0,1,5:42 PM,11:39,NaN,NaN,Bogut REBOUND (Off:0 Def:1),...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
4,41200161,4,2,1,1,5:42 PM,11:27,NaN,NaN,MISS Lee 15' Jump Shot,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0


Guardamos el dataset en un archivo nuevo de .CSV.
Se denominará 'juego_por_juego.csv'


In [12]:
play_by_play_filtrado.to_csv(r'C:\Users\Fernando\Desktop\CSV de NBA\archive\csv\juego_por_juego.csv', index=False)
#Se guarda el archivo nuevo ya con los id de juego pertenecientes al rango de fecha que especificamos.


Visualizacion de la tabla juego_por_juego


In [13]:
## mostramos el dataset filtrado
print(play_by_play_filtrado.shape)
play_by_play_filtrado.head(1000)

(5228811, 34)


,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,wctimestring,pctimestring,homedescription,neutraldescription,visitordescription,...,player2_team_nickname,player2_team_abbreviation,person3type,player3_id,player3_name,player3_team_id,player3_team_city,player3_team_nickname,player3_team_abbreviation,video_available_flag
0,41200161,0,12,0,1,5:41 PM,12:00,NaN,Start of 1st Period (5:41 PM EST),NaN,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
1,41200161,1,10,0,1,5:41 PM,12:00,Jump Ball Koufos vs. Bogut: Tip to Lawson,NaN,NaN,...,Warriors,GSW,4.0,201951,Ty Lawson,1.610613e+09,Denver,Nuggets,DEN,0
2,41200161,2,2,42,1,5:42 PM,11:40,MISS Fournier Driving Layup,NaN,Bogut BLOCK (1 BLK),...,NaN,NaN,5.0,101106,Andrew Bogut,1.610613e+09,Golden State,Warriors,GSW,0
3,41200161,3,4,0,1,5:42 PM,11:39,NaN,NaN,Bogut REBOUND (Off:0 Def:1),...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
4,41200161,4,2,1,1,5:42 PM,11:27,NaN,NaN,MISS Lee 15' Jump Shot,...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,41200111,135,1,1,2,3:42 PM,8:56,Shumpert 24' 3PT Jump Shot (3 PTS) (Kidd 1 AST),NaN,NaN,...,Knicks,NYK,0.0,0,NaN,NaN,NaN,NaN,NaN,0
996,41200111,136,9,1,2,3:42 PM,8:54,NaN,NaN,Celtics Timeout: Regular (Reg.2 Short 0),...,NaN,NaN,0.0,0,NaN,NaN,NaN,NaN,NaN,0
997,41200111,139,8,0,2,3:45 PM,8:54,NaN,NaN,SUB: Bradley FOR Crawford,...,Celtics,BOS,0.0,0,NaN,NaN,NaN,NaN,NaN,0
998,41200111,140,2,1,2,3:46 PM,8:31,Felton BLOCK (1 BLK),NaN,MISS Bradley 18' Jump Shot,...,NaN,NaN,4.0,101109,Raymond Felton,1.610613e+09,New York,Knicks,NYK,0


Observamos el tipo de datos:

In [14]:
for columna in play_by_play_filtrado.columns:
    print(columna)
    print(play_by_play_filtrado[columna].describe())
    print('------------------------------------')


game_id
count    5.228811e+06
mean     2.255060e+07
std      5.638654e+06
min      1.130000e+07
25%      2.150026e+07
50%      2.170121e+07
75%      2.200103e+07
max      4.220040e+07
Name: game_id, dtype: float64
------------------------------------
eventnum
count    5.228811e+06
mean     3.095331e+02
std      1.858390e+02
min      0.000000e+00
25%      1.500000e+02
50%      3.040000e+02
75%      4.570000e+02
max      1.018000e+03
Name: eventnum, dtype: float64
------------------------------------
eventmsgtype
count    5.228811e+06
mean     3.985255e+00
std      2.694789e+00
min      1.000000e+00
25%      2.000000e+00
50%      4.000000e+00
75%      5.000000e+00
max      1.800000e+01
Name: eventmsgtype, dtype: float64
------------------------------------
eventmsgactiontype
count    5.228811e+06
mean     1.457754e+01
std      2.771428e+01
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+00
75%      1.100000e+01
max      1.100000e+02
Name: eventmsgactiontype, dtype: float64

# ____________________________________________________________

Abrimos y exploramos el Dataset de Lesiones_2013-2023

In [67]:
## creamos variable para alojar la ruta del archivo
ruta_archivo_lesionados_2013_2023 = r'C:\Users\Fernando\Desktop\CSV de NBA\Lesiones_2013-2023.csv'

df_lesionados_1323 = pd.read_csv(ruta_archivo_lesionados_2013_2023)

print(df_lesionados_1323.head())
print("Cantidad de filas y columnas: ", df_lesionados_1323.shape)

   PLAYER_ID    PLAYER_NAME SEASON  SEASON_NUM   AGE PLAYER_HEIGHT_INCHES  \
0    1630639    A.J. Lawson  22-23        22.5  22.0                   78   
1    1631260       AJ Green  22-23        22.5  23.0                   77   
2    1631100     AJ Griffin  22-23        22.5  19.0                   78   
3     203932   Aaron Gordon  22-23        22.5  27.0                   80   
4    1628988  Aaron Holiday  22-23        22.5  26.0                   72   

  PLAYER_WEIGHT  GP   MIN  USG_PCT  ...  AVG_SEC_PER_TOUCH  \
0           179  13   4.8    0.182  ...               1.39   
1           190  32   9.4    0.160  ...               2.61   
2           220  66  19.4    0.174  ...               2.08   
3           235  61  30.1    0.206  ...               2.94   
4           185  57  13.7    0.128  ...               4.17   

   AVG_DRIB_PER_TOUCH  ELBOW_TOUCHES  POST_TOUCHES  PAINT_TOUCHES     TEAM  \
0                0.62            0.0           0.0            0.2      NaN   
1       

Vemos los tipos de datos y  estadisticas de cada columna.

In [43]:
for columna in df_lesionados_1323.columns:
    print(columna)
    print(df_lesionados_1323[columna].describe())
    print('------------------------------------')


PLAYER_ID
count    5.578000e+03
mean     7.789375e+05
std      7.189302e+05
min      7.080000e+02
25%      2.021300e+05
50%      2.035840e+05
75%      1.628399e+06
max      1.641645e+06
Name: PLAYER_ID, dtype: float64
------------------------------------
PLAYER_NAME
count             5578
unique            1389
top       Kemba Walker
freq                25
Name: PLAYER_NAME, dtype: object
------------------------------------
SEASON
count      5578
unique       10
top       21-22
freq        702
Name: SEASON, dtype: object
------------------------------------
SEASON_NUM
count    5578.000000
mean       18.241484
std         2.829290
min        13.500000
25%        15.500000
50%        18.500000
75%        20.500000
max        22.500000
Name: SEASON_NUM, dtype: float64
------------------------------------
AGE
count    5578.000000
mean       26.533883
std         4.253617
min        19.000000
25%        23.000000
50%        26.000000
75%        29.000000
max        43.000000
Name: AGE, dty

Guardamos dataset por si a caso.

In [70]:
df_lesionados_1323.to_csv(r'C:\Users\Fernando\Desktop\CSV de NBA\Lesiones_2013-2023.csv', index=False)


# ______________________________________________________________

Abrimos el dataset Lesiones_2010-2020


In [48]:
## creamos variable para alojar la ruta del archivo
ruta_archivo_lesionados_2010_2020 = r'C:\Users\Fernando\Desktop\CSV de NBA\Lesiones_2010-2020.csv'

df_lesionados_1020 = pd.read_csv(ruta_archivo_lesionados_2010_2020)

print(df_lesionados_1020.head())
print("Cantidad de filas y columnas: ", df_lesionados_1020.shape)

       Date     Team         Nombre Acquired   Relinquished  \
0  03-10-10    Bulls  Carlos Boozer      NaN  Carlos Boozer   
1  06-10-10  Pistons  Jonas Jerebko      NaN  Jonas Jerebko   
2  06-10-10  Pistons  Terrico White      NaN  Terrico White   
3  08-10-10  Blazers     Jeff Ayres      NaN     Jeff Ayres   
4  08-10-10     Nets    Troy Murphy      NaN    Troy Murphy   

                                               Notes      ID  
0  fractured bone in right pinky finger (out inde...    2430  
1      torn right Achilles tendon (out indefinitely)  201973  
2  broken fifth metatarsal in right foot (out ind...  202358  
3          torn ACL in right knee (out indefinitely)  201965  
4             strained lower back (out indefinitely)    2211  
Cantidad de filas y columnas:  (27105, 7)


Convertimos los datos de fecha a tipo fecha:

In [59]:
# Convertimos al formato fecha.
df_lesionados_1020['Date'] = pd.to_datetime(df_lesionados_1020['Date'], errors='coerce')
df_lesionados_1020['Date'] = pd.to_datetime(df_lesionados_1020['Date'], format='%y-%m-%d')


print(df_lesionados_1020['Date'].dtype)
print(df_lesionados_1020['Date'].head(10000)) 
print(df_lesionados_1020['Date'].isnull().sum())




datetime64[ns]
0      2010-03-10
1      2010-06-10
2      2010-06-10
3      2010-08-10
4      2010-08-10
          ...    
9995   2014-05-04
9996   2014-06-04
9997   2014-06-04
9998   2014-06-04
9999   2014-06-04
Name: Date, Length: 10000, dtype: datetime64[ns]
0


Vemos los tipos de datos y  estadisticas de cada columna.

In [60]:
for columna in df_lesionados_1020.columns:
    print(columna)
    print(df_lesionados_1020[columna].describe())
    print('------------------------------------')

Date
count                            27105
mean     2015-09-05 05:46:29.551743232
min                2010-01-11 00:00:00
25%                2013-04-01 00:00:00
50%                2015-11-18 00:00:00
75%                2017-12-27 00:00:00
max                2020-12-09 00:00:00
Name: Date, dtype: object
------------------------------------
Team
count     27103
unique       32
top       Spurs
freq       1163
Name: Team, dtype: object
------------------------------------
Nombre
count          27101
unique          1158
top       Kevin Love
freq             139
Name: Nombre, dtype: object
------------------------------------
Acquired
count            9542
unique           1111
top       Dwyane Wade
freq               54
Name: Acquired, dtype: object
------------------------------------
Relinquished
count          17560
unique          1156
top       Kevin Love
freq             101
Name: Relinquished, dtype: object
------------------------------------
Notes
count                 27105
uniqu

Filtramos el dataset y eliminamos las fechas anteriores a 2012-10-30

In [65]:
# Filtrar el DataFrame para que queden solo con las fechas desde el 30-10-2012 en adelante
df_lesionados_1020_filtrado = df_lesionados_1020[df_lesionados_1020['Date'] > '2012-10-30']

# Mostramos el dataset:
print(df_lesionados_1020_filtrado.head())
print(df_lesionados_1020_filtrado.shape)

           Date     Team            Nombre      Acquired      Relinquished  \
2917 2012-11-01    76ers     Spencer Hawes           NaN     Spencer Hawes   
2918 2012-11-01  Blazers      Luke Babbitt  Luke Babbitt               NaN   
2919 2012-11-01    Bulls       C.J. Watson           NaN       C.J. Watson   
2920 2012-11-01    Bulls      Derrick Rose           NaN      Derrick Rose   
2921 2012-11-01    Bulls  Richard Hamilton           NaN  Richard Hamilton   

                                      Notes      ID  
2917              strained lower back (DNP)  201150  
2918                              activated  202337  
2919  placed on IL with sprained left elbow  201228  
2920            sprained left big toe (DNP)  201565  
2921                  sore left groin (DNP)    1888  
(21923, 7)


Guardamos Dataset

In [66]:
df_lesionados_1020_filtrado.to_csv(r'C:\Users\Fernando\Desktop\CSV de NBA\Lesiones_2010-2020.csv', index=False)


# _________________________________________________________________

UNION DE lesiones_1020 y lesiones_1323 en un nuevo dataset, que se llamará: Lesiones

Abrimos ambos datasets.

In [82]:


# Cargar los archivos
df_2010_2020 = pd.read_csv(r'C:\Users\Fernando\Desktop\CSV de NBA\Lesiones_2010-2020.csv')
df_2013_2023 = pd.read_csv(r'C:\Users\Fernando\Desktop\CSV de NBA\Lesiones_2013-2023.csv')


# Revisamos los nombres de cada una de sus columnas.
print(df_2010_2020.columns)
print("\n")
print(df_2013_2023.columns)

print(df_2010_2020[['ID']].head())
print("\n")
print(df_2013_2023[['PLAYER_ID']].head())






Index(['Date', 'Team', 'Nombre', 'Acquired', 'Relinquished', 'Notes', 'ID'], dtype='object')


Index(['PLAYER_ID', 'PLAYER_NAME', 'SEASON', 'SEASON_NUM', 'AGE',
       'PLAYER_HEIGHT_INCHES', 'PLAYER_WEIGHT', 'GP', 'MIN', 'USG_PCT', 'PACE',
       'POSS', 'FGA_PG', 'DRIVES', 'DRIVE_FGA', 'DRIVE_PASSES', 'DIST_MILES',
       'AVG_SPEED', 'PULL_UP_FGA', 'PULL_UP_FG3A', 'TOUCHES',
       'FRONT_CT_TOUCHES', 'AVG_SEC_PER_TOUCH', 'AVG_DRIB_PER_TOUCH',
       'ELBOW_TOUCHES', 'POST_TOUCHES', 'PAINT_TOUCHES', 'TEAM', 'INJURED ON',
       'RETURNED', 'DAYS MISSED', 'INJURED_TYPE'],
      dtype='object')
       ID
0  201150
1  202337
2  201228
3  201565
4    1888


   PLAYER_ID
0    1630639
1    1631260
2    1631100
3     203932
4    1628988


Convertimos los Datos de ID y PLAYER_ID en str, porque puede haber valores no numericos o raro.

In [84]:
df_2010_2020['ID'] = df_2010_2020['ID'].astype(str)
df_2013_2023['PLAYER_ID'] = df_2013_2023['PLAYER_ID'].astype(str)



Vamos a unir ambos dataset a travez de ID y PLAYER_ID

In [85]:
# Hacemos el merge por ID de ambos datasets
# how='inner': solo quedarán los registros que tengan coincidencia en ambos datasets.
# Si querés mantener todos los de uno y solo los que coincidan del otro, usá left o right.

df_unido = pd.merge(
    df_2010_2020, 
    df_2013_2023, 
    left_on='ID', 
    right_on='PLAYER_ID',
    how='inner'  # Cambiá a 'outer', 'left' o 'right' según lo que quieras lograr
)

#Mostramos
print(df_unido.shape)
print(df_unido.head())


(143117, 39)
         Date     Team         Nombre      Acquired   Relinquished  \
0  2012-11-01    76ers  Spencer Hawes           NaN  Spencer Hawes   
1  2012-11-01    76ers  Spencer Hawes           NaN  Spencer Hawes   
2  2012-11-01    76ers  Spencer Hawes           NaN  Spencer Hawes   
3  2012-11-01    76ers  Spencer Hawes           NaN  Spencer Hawes   
4  2012-11-01  Blazers   Luke Babbitt  Luke Babbitt            NaN   

                       Notes      ID PLAYER_ID    PLAYER_NAME SEASON  ...  \
0  strained lower back (DNP)  201150    201150  Spencer Hawes  16-17  ...   
1  strained lower back (DNP)  201150    201150  Spencer Hawes  15-16  ...   
2  strained lower back (DNP)  201150    201150  Spencer Hawes  14-15  ...   
3  strained lower back (DNP)  201150    201150  Spencer Hawes  13-14  ...   
4                  activated  202337    202337   Luke Babbitt  17-18  ...   

   AVG_SEC_PER_TOUCH  AVG_DRIB_PER_TOUCH ELBOW_TOUCHES POST_TOUCHES  \
0               1.77            

Vemos valores no numericos en ID


In [86]:
mask_no_numericos = ~df_2010_2020['ID'].astype(str).str.isnumeric()
print(df_2010_2020[mask_no_numericos]['ID'].value_counts())

ID
NoEncontrado    20
Name: count, dtype: int64


Vemos los tipos de datos:


In [87]:
for columna in df_unido.columns:
    print(columna)
    print(df_unido[columna].describe())
    print('------------------------------------')

Date
count         143117
unique          1715
top       2017-12-04
freq             594
Name: Date, dtype: object
------------------------------------
Team
count       143113
unique          31
top       Warriors
freq          6243
Name: Team, dtype: object
------------------------------------
Nombre
count           143117
unique             939
top       Derrick Rose
freq              2185
Name: Nombre, dtype: object
------------------------------------
Acquired
count            54280
unique             911
top       Derrick Rose
freq               805
Name: Acquired, dtype: object
------------------------------------
Relinquished
count            88843
unique             936
top       Derrick Rose
freq              1380
Name: Relinquished, dtype: object
------------------------------------
Notes
count                143117
unique                 2546
top       activated from IL
freq                  37506
Name: Notes, dtype: object
------------------------------------
ID
count     1

Guardamos Dataset como Lesiones.csv

In [88]:
df_unido.to_csv(r'C:\Users\Fernando\Desktop\CSV de NBA\Lesiones.csv', index=False)
